In [117]:
import numpy as np
import pandas as pd

### Load parameter

In [118]:
sheet = "Problem 3 Instance 2"
file = "OR112-2_hw02_data.xlsx"

m = pd.read_excel(file,usecols="B:B",nrows=1,sheet_name=sheet)
m = int(m.columns.item())
n = pd.read_excel(file,usecols="B:B",skiprows=range(1),nrows=1,sheet_name=sheet)
n = int(n.columns.item())
p = pd.read_excel(file,usecols="B:B",skiprows=range(2),nrows=1,sheet_name=sheet)
p = int(p.columns.item())

d = pd.read_excel(file,usecols="E:"+chr(ord('E')+n-1),skiprows=range(4),nrows=m,sheet_name=sheet)
d = d.to_numpy()

h = pd.read_excel(file,usecols="B:B",skiprows=range(4),nrows=m,sheet_name=sheet)
h = h.to_numpy()

## Algorithm
### Greedy

In [119]:
def objfunc(h,d,x):
    total = 0
    for i in range(h.shape[0]):
        total += h[i] * min(d[i,x.astype(bool)])
    return total.item()



In [120]:
x = np.ones(n)
for i in range(n-p):
    total = np.zeros(n)
    for j in range(n-i):
        tmp_x = x.copy()
        if tmp_x[j] == 1:
            tmp_x[j] = 0
            total[j] = objfunc(h,d,tmp_x)
    x[np.argmax(total)] = 0
print(x)
print(objfunc(h,d,x))


[0. 1. 0. 1. 0. 1. 0. 0. 1. 1.]
65364


In [121]:
for i in range(n):
    print(f"x_{i+1} = {int(x[i])}", end=', ')
print()
for j in range(m):
    print(f"w_{j+1} = {min(d[j,x.astype(bool)])}", end=', ')

x_1 = 0, x_2 = 1, x_3 = 0, x_4 = 1, x_5 = 0, x_6 = 1, x_7 = 0, x_8 = 0, x_9 = 1, x_10 = 1, 
w_1 = 48, w_2 = 37, w_3 = 67, w_4 = 45, w_5 = 87, w_6 = 55, w_7 = 76, w_8 = 108, w_9 = 26, w_10 = 107, w_11 = 37, w_12 = 121, w_13 = 29, w_14 = 88, w_15 = 56, w_16 = 85, w_17 = 100, w_18 = 71, w_19 = 7, w_20 = 23, 

### LR

In [122]:
from gurobipy import *
import numpy as np
import pandas as pd

sheet = "Problem 3 Instance 2"
file = "OR112-2_hw02_data.xlsx"
model = Model(sheet)

m = pd.read_excel(file,usecols="B:B",nrows=1,sheet_name=sheet)
m = int(m.columns.item())
n = pd.read_excel(file,usecols="B:B",skiprows=range(1),nrows=1,sheet_name=sheet)
n = int(n.columns.item())
p = pd.read_excel(file,usecols="B:B",skiprows=range(2),nrows=1,sheet_name=sheet)
p = int(p.columns.item())

d = pd.read_excel(file,usecols="E:"+chr(ord('E')+n-1),skiprows=range(4),nrows=m,sheet_name=sheet)
d = d.to_numpy()

h = pd.read_excel(file,usecols="B:B",skiprows=range(4),nrows=m,sheet_name=sheet)
h = h.to_numpy()

x = model.addVars(n,lb=0.0,ub=1.0,vtype=GRB.CONTINUOUS, name="x")
w = model.addVars(m,vtype=GRB.CONTINUOUS, name="w")

model.setObjective(quicksum(h[i][0]*w[i] for i in range(m)), GRB.MAXIMIZE)

model.addConstr(quicksum(x[i] for i in range(n)) >= p)
for i in range(m):
    for j in range(n):
        model.addConstr(w[i] <= d[i][j]*x[j]+np.max(d[i])*(1-x[j]))

model.optimize()
x = []
# for v in model.getVars():
#     print('%s %g' % (v.varName, v.x))
#     x.append(v.x)

for v in model.getVars():
    print(f"&{v.varName.replace('[','_{').replace(']','}')} = {v.x}", end=',\\\\ ')
    x.append(v.x)

x = x[:n]

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (linux64 - "EndeavourOS")

CPU model: Intel(R) Core(TM) i5-10400F CPU @ 2.90GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 201 rows, 30 columns and 389 nonzeros
Model fingerprint: 0x63863c25
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+00, 2e+02]
Presolve removed 21 rows and 0 columns
Presolve time: 0.00s
Presolved: 180 rows, 30 columns, 368 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.8780500e+05   8.000000e+01   0.000000e+00      0s
      69    1.1781042e+05   0.000000e+00   0.000000e+00      0s

Solved in 69 iterations and 0.00 seconds (0.00 work units)
Optimal objective  1.178104228e+05
&x_{0} = 0.47636549575898146,\\ &x_{1} = 0.444908809010339,\\ &x_{2} = 0.433059541599074,\\ &x_{3} = 0.439008161675

In [123]:
tmp_x = np.zeros(n)
print(x)
for i in range(p):
    tmp_x[np.argmax(x)] = 1
    x[np.argmax(x)] = 0

for i in range(n):
    print(f"&x_{i+1} = {int(tmp_x[i])}", end=',\\\\\n ')
for i in range(m):
    print(f"w_{{{i+1}}} = {min(d[i,tmp_x.astype(bool)])}", end=', ')
print(objfunc(h,d,tmp_x))

[0.47636549575898146, 0.444908809010339, 0.433059541599074, 0.4390081616759844, 0.6125695279199782, 0.5543705001160574, 0.47559217514898305, 0.36340660833488725, 0.4909176989787073, 0.7098014814570079]
&x_1 = 1,\\
 &x_2 = 0,\\
 &x_3 = 0,\\
 &x_4 = 0,\\
 &x_5 = 1,\\
 &x_6 = 1,\\
 &x_7 = 0,\\
 &x_8 = 0,\\
 &x_9 = 1,\\
 &x_10 = 1,\\
 w_{1} = 79, w_{2} = 71, w_{3} = 50, w_{4} = 63, w_{5} = 87, w_{6} = 55, w_{7} = 71, w_{8} = 41, w_{9} = 53, w_{10} = 62, w_{11} = 37, w_{12} = 27, w_{13} = 91, w_{14} = 81, w_{15} = 77, w_{16} = 30, w_{17} = 76, w_{18} = 38, w_{19} = 38, w_{20} = 23, 63044
